In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time # for sleep time
import csv
import re
import functools
import datetime #for csv file title

driver = webdriver.Chrome(r'C:\Users\priya\chromedriver.exe')

# Tell Selenium to get the URL you're interested in.
driver.get('https://www.hotels.com/search.do?resolved-location=CITY%3A1404711%3AUNKNOWN%3AUNKNOWN&destination-id=1404711&q-destination=Orlando,%20Florida,%20United%20States%20of%20America&q-rooms=1&q-room-0-adults=3&q-room-0-children=0')




def reduce_concat(x, sep=""):
    return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

def paste(*lists, sep=" ", collapse=None):
    result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
    if collapse is not None:
        return reduce_concat(result, sep=collapse)
    return list(result)

def scroll_bar():
    #scroll to end of page so that all elements are visible
    calm = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//html"))) #if elements aren't available it will return a timeout error and alert user that script must be altered
    time.sleep(3)
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False

    while(match==False):
        lastCount = lenOfPage
        time.sleep(3)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True

    scroll = driver.find_element_by_xpath('//html')
    time.sleep(2)
    scroll.send_keys(Keys.HOME)
    time.sleep(2)


#collect html tags, this will need to go inside of the nested loop function
wait_event = WebDriverWait(driver, 10)
events = wait_event.until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="description resp-module"]')))
lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    
hotel_all_links = []
newPos = 0
while (newPos <= lenOfPage):
    print('Number of events:', len(events))
    print('position pre-loop:', newPos) #for testing
    scroll_bar()
    time.sleep(3)

    offprep = paste(["var offprep = window.screen.height * "], [(len(events))/5], [";return offprep"], sep='') 
    offprep = ''.join(offprep)
    offset = driver.execute_script(offprep)
    newPos = newPos + offset
    scrollerArg = paste(["window.scrollTo(0, "], [newPos], [");"], sep = '')
    scrollerArg = ''.join(scrollerArg)
    time.sleep(20)
    scroller = driver.execute_script(scrollerArg)
    events = wait_event.until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="description resp-module"]')))

    print('Number of events:', len(events)) #testing
    print('current position:', newPos) #testing

# Get list of links to hotels and append   
time.sleep(40)
hotel_list = (driver.find_elements_by_xpath('//div[@class="description resp-module"]/figure[@class="image"]/a'))
hotel_all_links = [hotel.get_attribute('href') for hotel in hotel_list ] 

print(len(hotel_all_links))
print('+'*50)
print(hotel_all_links)
time.sleep(4)


time.sleep(1.5)

#Open CSV Writer
now = datetime.datetime.now() #for csv file title
filename = paste(['HotelsOrl'], [now.year], ['-'], [now.month], ['-'], [now.day],['.csv'], sep='')
filename = ''.join(filename)
csv_file = open(filename, 'w', encoding='utf-8',newline = '')
writer = csv.writer(csv_file)


#enter the url of hotel after gathering all links and get the variables

for url in hotel_all_links:
    # Initialize an empty dictionary for each review
    #dictionary to store hotel data
    hotel_dict = {}
    try:
        driver.get(url)
        print('+'*50)
        print(url)
    except:
        continue
    try:
        time.sleep(1.5)
        try:
            hotel_title = driver.find_element_by_xpath('//div[@class="vcard"]/h1').text
        except Exception as e :
            print("issue in hotel title")
            print(e)
            hotel_title = None

        try:
            hotel_type =  driver.find_element_by_xpath('//div[@class="widget-query-group widget-query-destination"]/input').get_attribute('data-as-type')
        except:
            print("issue in hotel type")
            hotel_type = None

        try:    
            hotel_price = driver.find_element_by_xpath('//span[@class="current-price"]').text
        except:
            try:
                hotel_price = driver.find_element_by_xpath('//div[@class="price"]/span[@class="current-price has-old-price"]').text
            except:
                try:
                    hotel_price = driver.find_element_by_xpath('//div[@class="price"]/span[@class="current-price has-old-price bold"]').text
                except:
                    print("issue in hotel_price")    
                    hotel_price = None


        try:
            hotel_star = driver.find_element_by_xpath('//span[@class="star-rating-text star-rating-text-strong widget-star-rating-overlay"]').text

        except:
            try:
                hotel_star = driver.find_element_by_xpath('//span[@class="star-rating-text widget-star-rating-overlay"]').text

            except:
                print("issue in hotel star")
                hotel_star = None

        try:
            no_of_ratings = driver.find_element_by_xpath('//div/a[@class="total-reviews trust-you"]').text

        except:
            try:
                no_of_ratings = driver.find_element_by_xpath('//div[@class="ta-total-reviews"]').text

            except Exception as e :
                print("issue in no_of_ratings")
                print(e)
                no_of_raf_ratings = None  

        time.sleep(2)        
        try:
            coord_details = driver.find_elements_by_xpath('//span[@class="hotel-coordinates"]/meta')

            lat_long = [detail.get_attribute('content') for detail in coord_details ]

        except:
            print("Issue in lat_long")
            lat_long = None

        time.sleep(2.5)
        try:
            rating = driver.find_element_by_xpath('//div/span[@class="rating"]').text

        except:
            print("issue in rating")
            rating = None
            
        time.sleep(1)
        try:
            occupancy = driver.find_element_by_xpath('//div/span[@class="search-params"]').text

        except:
            print("issue in occupancy")
            occupancy = None
            
        hotel_dict['hotel_title'] = hotel_title
        hotel_dict['hotel_type'] = hotel_type
        hotel_dict['hotel_price'] = hotel_price
        hotel_dict['hotel_star'] = hotel_star
        hotel_dict['no_of_ratings'] = no_of_ratings
        hotel_dict['rating'] = rating
        hotel_dict['occupancy'] = occupancy
        hotel_dict['lat_long'] = lat_long
        
        time.sleep(2)
        print("writing")
        writer.writerow(hotel_dict.values())

        print(hotel_title)
        print(hotel_type)
        print(hotel_price)
        print(hotel_star)
        print(no_of_ratings)
        print(rating)
        print(occupancy)
        print(lat_long)
    
    
    except Exception as e:
        print(e)
        continue
        
csv_file.close()
driver.close()


Number of events: 6
position pre-loop: 0
Number of events: 46
current position: 1036.8
Number of events: 46
position pre-loop: 1036.8
Number of events: 176
current position: 8985.599999999999
406
++++++++++++++++++++++++++++++++++++++++++++++++++
['https://www.hotels.com/travelads/trackredirect.html?trackingUrl=H4sIAAAAAAAAAD2Ux66sZhCE3-Zs8D2EIQxXGlkMmYEZctpYZH5yTk_vY1vypltdpVZtPlW5LMP8G4aXKdqyJkrn77pPU_Cd9C1s_6sx6Wxl0waSDN5wWOqXrJn_t9gGJPWfP89JDbqCi5bowbbDL_VchdfSvnE0wIgDA_lpRUoxsIL7uqvmzo7pVeC5Or2D00POsmU9ts3zzAON6e0nxB5VKYNl5-vrIvS1gjNtJm7wlNCuNoRASkgIEi4R9XzlLQaC7QipS80Z5SvV2mUXxB2djW-7TvQqltktPYDDfxf2SoN0xsOl1W2q9Yql2_nKalRuObZrqj87tyDK4WtkV5jLPrV1yxc4lEs7Ei4SiM4i30ZrjihNvrW58VSewX5LP5D5dgJm2qOrkuwEobkgyXrmXgUca3l7udK-kK1xrENsVd3FjiSrSGZZKdtw3NT0_sNjOtfp7ouPJs7vIIzJTcaRCjuKXgSEfmgh3aZkQycmmG_hArpNveqYgtxF8WU6dn0Lm-UqOoita3APU-NuvRnxCNVjg-3IpChGMIiCnFwqmyOY5c8ZoYZ3un1btwKegpEWZHQQd9aVXjeogFXaJvk3OLhVKPkgeQFUL2vhaXKr3mybkSs0QrwcX82vjkcQWoXjFJ542M3Fnn8aSqF9_qGEYQWFKAKySJ_OkPqn4qzj7jk

++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/travelads/trackredirect.html?trackingUrl=H4sIAAAAAAAAAD2Ux66sZhCE3-Zs8D2EIQxXGlkMmYEZctpYZH5yTk_vY1vypltdpVZtPlW5LMP8G4aXKdqyJkrn77pPU_Cd9C1s_6sx6Wxl0waSDN5wWOqXrJn_t9gGJPWfP89JDbqCi5bowbbDL_VchdfSvnE0wIgDA_lpRUoxsIL7uqvmzo7pVeC5Or2D00POsmU9ts3zzAON6e0nxB5VKYNl5-vrIvS1gjNtJm7wlNCuNoRASkgIEi4R9XzlLQaC7QipS80Z5SvV2mUXxB2djW-7TvQqltktPYDDfxf2SoN0xsOl1W2q9Yql2_nKalRuObZrqj87tyDK4WtkV5jLPrV1yxc4lEs7Ei4SiM4i30ZrjihNvrW58VSewX5LP5D5dgJm2qOrkuwEobkgyXrmXgUca3l7udK-kK1xrENsVd3FjiSrSGZZKdtw3NT0_sNjOtfp7ouPJs7vIIzJTcaRCjuKXgSEfmgh3aZkQycmmG_hArpNveqYgtxF8WU6dn0Lm-UqOoita3APU-NuvRnxCNVjg-3IpChGMIiCnFwqmyOY5c8ZoYZ3un1btwKegpEWZHQQd9aVXjeogFXaJvk3OLhVKPkgeQFUL2vhaXKr3mybkSs0QrwcX82vjkcQWoXjFJ542M3Fnn8aSqF9_qGEYQWFKAKySJ_OkPqn4qzj7jk2UGZFw0-O_ShN8flkc4trFeun412Pn-N5W16fotRvyuhLp4_wPDn6SUQ7lCc5Ps617-xTMZTZ85hfV-7Thd_c-YKARfvaG9V2LwzxkHrP-fqR3GoTebMad3V8hrJIDpJpX4GRo4sc4X7iUs4O5jkmN_eHx0lnuoHHt4-GZXy4X9txAM0s23NqZLCOEPoscQOedqr_C

++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho129374/?pa=23&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Four Points by Sheraton Orlando International Drive, Orlando
HOTEL
$101
3.5-star
See all 627 Hotels.com reviews
8.0
1 room, 3 adults
['28.462304', '-81.461441']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho204436/?pa=24&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Legacy Vacation Resorts-Orlando, Kissimmee
HOTEL
$66
3-star
See all 1,487 Hotels.com reviews
7.4
1 room, 3 adults
['28.329881', '-81.489634']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho121146/?pa=25&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Courtyard by Marriott Orlando Airport, Orlando
HOTEL
$74
3-star
See all 316 Hotels.com reviews
8.4
1 room, 3 adults
['28.459671', '-81.312231']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ww

writing
Rosen Inn at Pointe Orlando, Orlando
HOTEL
$72
3-star
See all 2,033 Hotels.com reviews
8.2
1 room, 3 adults
['28.433898', '-81.47213']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho109569/?pa=50&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Disney's Old Key West Resort, Lake Buena Vista
HOTEL
$540
3.5-star
See all 36 Hotels.com reviews
9.0
1 room, 3 adults
['28.375826', '-81.533339']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho124122/?pa=51&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Quality Suites, Orlando
HOTEL
$71
2.5-star
See all 808 Hotels.com reviews
6.4
1 room, 3 adults
['28.430306', '-81.475095']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho106105/?pa=52&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Disney's Caribbean Beach Resort, Lake Buena Vista
HOTEL
$204
3.5-star
See all 621 Hotels.com rev

++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho109476/?pa=76&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Days Inn by Wyndham Orlando Near Millenia Mall, Orlando
HOTEL
$44
2-star
See all 1,796 Hotels.com reviews
4.2
1 room, 3 adults
['28.508172', '-81.416658']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho119710/?pa=77&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Ramada by Wyndham Orlando Near Convention Center, Orlando
HOTEL
$60
2.5-star
See all 1,004 Hotels.com reviews
5.8
1 room, 3 adults
['28.444862', '-81.472482']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho200211/?pa=78&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Parc Corniche Condominium Suite Hotel, Orlando
HOTEL
$99
3-star
See all 667 Hotels.com reviews
7.8
1 room, 3 adults
['28.399183', '-81.460788']
++++++++++++++++++++++++++++++++++++++++++++++++++

...

++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho205052/?pa=307&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Crowne Plaza Orlando Universal, Orlando
HOTEL
$108
4-star
See all 525 Hotels.com reviews
8.2
1 room, 3 adults
['28.451459', '-81.465307']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho226130/?pa=308&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
issue in hotel_price
writing
Maingate Magic Vacation Homes by IPG, Davenport
HOTEL
None
3.5-star
From 301 reviews
TripAdvisor Traveller Rating
7.2
1 room, 3 adults
['28.297446', '-81.665579']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho299125/?pa=309&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Barefoot N Resort by Diamond Resorts, Kissimmee
HOTEL
$94
3-star
See all 543 Hotels.com reviews
9.0
1 room, 3 adults
['28.329264', '-81.516597']
++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho450958/?pa=332&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Rodeway Inn Kissimmee Maingate West, Kissimmee
HOTEL
$33
2-star
See all 414 Hotels.com reviews
5.6
1 room, 3 adults
['28.33267', '-81.58519']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho220193/?pa=333&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Omni Orlando Resort at ChampionsGate, Davenport
HOTEL
$135
4.5-star
See all 703 Hotels.com reviews
9.0
1 room, 3 adults
['28.261267', '-81.623209']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho450680/?pa=334&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Motel 6 Orlando - Winter Park, Orlando
HOTEL
$60
2-star
See all 215 Hotels.com reviews
5.6
1 room, 3 adults
['28.60765', '-81.39306']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho17119

issue in rating
writing
Windsor Palms by Global Resort Homes, Kissimmee
HOTEL
None
3.5-star
See all 84 Hotels.com reviews
None
1 room, 3 adults
['28.318788', '-81.607218']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho217884/?pa=355&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
issue in hotel_price
writing
Florida Vacation Villas, Kissimmee
HOTEL
None
3-star
From 256 reviews
TripAdvisor Traveller Rating
Good7.0
1 room, 3 adults
['28.328096', '-81.489457']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho468703/?pa=356&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
issue in hotel_price
writing
Windsor Palms Resort by Optimal Growth, Kissimmee
HOTEL
None
3.5-star
See all 25 Hotels.com reviews
8.0
1 room, 3 adults
['28.315946', '-81.610816']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho236632/?pa=357&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=

issue in hotel_price
writing
Caribe Cove Resort by Wyndham Vacation Rentals, Kissimmee
HOTEL
None
3.5-star
See all 748 Hotels.com reviews
8.8
1 room, 3 adults
['28.337935', '-81.64869']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho256787/?pa=380&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
The Berkley, Orlando (No Resort Fees) , Kissimmee
HOTEL
$152
4-star
See all 400 Hotels.com reviews
9.2
1 room, 3 adults
['28.347078', '-81.627948']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho266244/?pa=381&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Melia Orlando Suite Hotel at Celebration, Celebration
HOTEL
$125
4-star
See all 1,204 Hotels.com reviews
8.2
1 room, 3 adults
['28.329354', '-81.535804']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho260008/?pa=382&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Hampton Inn & Sui

writing
Homewood Suites by Hilton Lake Buena Vista Orlando, Orlando
HOTEL
$105
3-star
See all 450 Hotels.com reviews
8.6
1 room, 3 adults
['28.39288', '-81.493243']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho524755/?pa=405&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
WoodSpring Suites Orlando South, Orlando
HOTEL
$90
2.5-star
See all 155 Hotels.com reviews
5.6
1 room, 3 adults
['28.414994', '-81.421937']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho298444/?pa=406&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
issue in hotel_price
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="ta-total-reviews"]"}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

writing
Magic Landing, Kissimmee
HOTEL
None
3-sta